In [1]:
import os
import json
import hashlib

# Đường dẫn đến thư mục gốc
root_dir = "D:\\AI\\WebCrawler\\tiki"

# Tạo biến đếm ID người dùng
user_id_counter = 1

# Tạo một danh sách để lưu trữ dữ liệu người dùng
user_data = []

# Tạo một set để lưu trữ các khóa người dùng duy nhất
unique_user_names = set()

# Duyệt qua tất cả các thư mục con trong thư mục gốc
for dir_name in os.listdir(root_dir):
    dir_path = os.path.join(root_dir, dir_name)
    if os.path.isdir(dir_path):
        # Duyệt qua tất cả các tệp tin trong thư mục con
        for file_name in os.listdir(dir_path):
            file_path = os.path.join(dir_path, file_name)
            if file_name.endswith(".json"):
                # Đọc dữ liệu từ tệp tin JSON
                with open(file_path, "r", encoding="utf-8") as f:
                    data = json.load(f)

                # Lấy thông tin người dùng từ dữ liệu
                for review in data["reviews"]:
                    reviewer = review["reviewer"]
                    name = reviewer["name"]

                    # Kiểm tra xem người dùng đã tồn tại trong set hay chưa
                    if name not in unique_user_names:
                        unique_user_names.add(name)

                        # Lưu trữ dữ liệu người dùng vào danh sách
                        user_data.append(
                            {
                                "id": user_id_counter,
                                "name": name,
                                "date": reviewer["date"],
                                "review_count": int(
                                    reviewer["review_count"].split(" ")[0]
                                ),
                                "liked_count": int(
                                    reviewer["liked_count"].split(" ")[0]
                                ),
                                "categories": [dir_name],
                            }
                        )

                        # Tăng biến đếm ID người dùng
                        user_id_counter += 1

# Ghi dữ liệu người dùng vào tệp tin JSON
with open("user_data.json", "w", encoding="utf-8") as jsonfile:
    json.dump(user_data, jsonfile, ensure_ascii=False, indent=4)

In [2]:
import csv

# Đọc dữ liệu từ file CSV
with open("user_data_all.csv", "r", encoding="utf-8") as csvfile:
    reader = csv.DictReader(csvfile)
    user_data_all = list(reader)

# Đọc dữ liệu từ file CSV
with open("user_data.csv", "r", encoding="utf-8") as csvfile:
    reader = csv.DictReader(csvfile)
    user_data = list(reader)

print(f"Tổng số người dùng khi kiểm tra đầy đủ thông tin: {len(user_data_all)}")
print(f"Tổng số người dùng khác tên: {len(user_data)}")

Tổng số người dùng khi kiểm tra đầy đủ thông tin: 89168
Tổng số người dùng khác tên: 62138


In [2]:
import os
import json

# Đường dẫn đến thư mục gốc
root_dir = "D:\\AI\\WebCrawler\\tiki"

# Tạo biến đếm ID item
item_id_counter = 1

# Tạo một danh sách để lưu trữ dữ liệu về sách
item_data = []

# Duyệt qua tất cả các thư mục con trong thư mục gốc
for dir_name in os.listdir(root_dir):
    dir_path = os.path.join(root_dir, dir_name)
    if os.path.isdir(dir_path):
        # Duyệt qua tất cả các tệp tin trong thư mục con
        for file_name in os.listdir(dir_path):
            file_path = os.path.join(dir_path, file_name)
            if file_name.endswith(".json"):
                # Lấy ID từ biến đếm
                file_id = str(item_id_counter)

                # Đọc dữ liệu từ tệp tin JSON
                with open(file_path, "r", encoding="utf-8") as f:
                    data = json.load(f)

                # Lấy thông tin về sách từ dữ liệu
                title = data["info"]["title"]
                author = data["info"]["author"]
                details = data["details"]
                rating = data.get("rating", {})
                total_rating = rating.get("total_rating", 0)
                number_of_rating = rating.get("number_of_rating", 0)
                publisher = details.get("Nhà xuất bản", "")
                cover_type = details.get("Loại bìa", "")
                page_count = details.get("Số trang", "")

                # Lưu trữ thông tin sách vào danh sách
                item_data.append(
                    {
                        "id": file_id,
                        "title": title,
                        "author": author,
                        "total_rating": total_rating,
                        "number_of_rating": number_of_rating,
                        "publisher": publisher,
                        "cover_type": cover_type,
                        "page_count": page_count,
                        "categories": [dir_name],
                    }
                )

                # Tăng biến đếm ID item
                item_id_counter += 1

# Ghi dữ liệu sách vào tệp tin JSON
with open("item_data.json", "w", encoding="utf-8") as jsonfile:
    json.dump(item_data, jsonfile, ensure_ascii=False, indent=4)

In [4]:
import csv

# Đọc dữ liệu từ file CSV
with open("item_data.csv", "r", encoding="utf-8") as csvfile:
    reader = csv.DictReader(csvfile)
    item_data = list(reader)

# Hiển thị tổng số lượng item
print(f"Tổng số lượng item: {len(item_data)}")

Tổng số lượng item: 234


In [5]:
import csv

# Khởi tạo một từ điển để lưu trữ số lượng item cho mỗi thể loại
category_counts = {}

# Đọc dữ liệu từ file CSV
with open("item_data.csv", "r", encoding="utf-8") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        categories = row["categories"].split(
            ","
        )  # Chuyển đổi danh sách thể loại thành danh sách các chuỗi
        for category in categories:
            if category in category_counts:
                category_counts[category] += 1
            else:
                category_counts[category] = 1

# Hiển thị số lượng item cho mỗi thể loại
print("Thống kê số lượng item theo từng thể loại:")
for category, count in category_counts.items():
    print(f"{category}: {count}")

Thống kê số lượng item theo từng thể loại:
khoa-hoc-ky-thuat: 54
sach-kien-thuc-tong-hop: 54
sach-kinh-te: 16
sach-van-hoc: 52
tam-ly-gioi-tinh: 5
trinh-tham: 53


In [3]:
import os
import json

# Đường dẫn đến thư mục gốc
root_dir = "D:\\AI\\WebCrawler\\tiki"

# Tạo một từ điển để lưu trữ ánh xạ tên người dùng sang user_id
name_to_user_id = {}

# Tạo một từ điển để lưu trữ ánh xạ tên sách sang item_id
title_to_item_id = {}

# Đọc dữ liệu từ file JSON user_data
with open("user_data.json", "r", encoding="utf-8") as jsonfile:
    user_data = json.load(jsonfile)
    for user in user_data:
        name_to_user_id[user["name"]] = user["id"]

# Đọc dữ liệu từ file JSON item_data
with open("item_data.json", "r", encoding="utf-8") as jsonfile:
    item_data = json.load(jsonfile)
    for item in item_data:
        title_to_item_id[item["title"]] = item["id"]

# Tạo một danh sách để lưu trữ dữ liệu rating
rating_data = []

# Duyệt qua tất cả các thư mục con trong thư mục gốc
for dir_name in os.listdir(root_dir):
    dir_path = os.path.join(root_dir, dir_name)
    if os.path.isdir(dir_path):
        # Duyệt qua tất cả các tệp tin trong thư mục con
        for file_name in os.listdir(dir_path):
            file_path = os.path.join(dir_path, file_name)
            if file_name.endswith(".json"):
                # Đọc dữ liệu từ tệp tin JSON
                with open(file_path, "r", encoding="utf-8") as f:
                    data = json.load(f)
                    info = data["info"]
                    title = info["title"]
                    item_id = title_to_item_id.get(title, "")

                    # Lặp qua tất cả các review và lưu vào danh sách rating_data
                    for review in data["reviews"]:
                        reviewer = review["reviewer"]
                        name = reviewer["name"]
                        user_id = name_to_user_id.get(name, "")
                        rating = review["review"]["rating"]
                        content = review["review"]["content"]
                        rating_data.append(
                            {
                                "user_id": user_id,
                                "item_id": item_id,
                                "rating": rating,
                                "content": content,
                            }
                        )

# Ghi dữ liệu rating vào tệp tin JSON
with open("rating.json", "w", encoding="utf-8") as jsonfile:
    json.dump(rating_data, jsonfile, ensure_ascii=False, indent=4)

print("File rating.json đã được tạo thành công.")

File rating.json đã được tạo thành công.


In [4]:
import json

# Đọc dữ liệu từ file JSON vào danh sách
rating_data = []
with open("rating.json", "r", encoding="utf-8") as jsonfile:
    rating_data = json.load(jsonfile)

# Sắp xếp danh sách theo user_id
rating_data.sort(key=lambda x: x["user_id"])

# Ghi lại dữ liệu đã sắp xếp vào file JSON mới
with open("rating_sorted.json", "w", encoding="utf-8") as jsonfile:
    json.dump(rating_data, jsonfile, ensure_ascii=False, indent=4)

print("Dữ liệu đã được sắp xếp theo user_id và ghi vào file rating_sorted.json.")

Dữ liệu đã được sắp xếp theo user_id và ghi vào file rating_sorted.json.


In [4]:
import json

# Đọc dữ liệu từ file JSON
rating_data = []
with open("rating_sorted.json", "r", encoding="utf-8") as jsonfile:
    rating_data = json.load(jsonfile)

# Tính toán số lượng mẫu trong tập huấn luyện và tập kiểm tra
total_samples = len(rating_data)
train_ratio = 0.8  # Tỷ lệ mẫu trong tập huấn luyện
train_size = int(train_ratio * total_samples)

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
train_data = rating_data[:train_size]
test_data = rating_data[train_size:]


# Ghi dữ liệu vào các file JSON tương ứng
def write_data_to_json(data, filename):
    with open(filename, "w", encoding="utf-8") as jsonfile:
        json.dump(data, jsonfile, ensure_ascii=False, indent=4)


write_data_to_json(train_data, "train_data.json")
write_data_to_json(test_data, "test_data.json")
print("Dữ liệu đã được chia thành tập huấn luyện và tập kiểm tra.")

Dữ liệu đã được chia thành tập huấn luyện và tập kiểm tra.
